In [2]:
from trainer_encodec_vc_inference import run
from types import SimpleNamespace

args = SimpleNamespace(
    dataset="lca0503/soxdata_small_encodec",
    splits=["train"],
    ground_truth_only=False,
    cascade_ar_nar=True,
    nar_model_only=False,
    ground_truth_model_name="voidful/bart-base-unit",
    ar_checkpoint="lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans",
    nar_checkpoint="lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans",
    ground_truth_output_path="output_wav/vc/ground_truth/train_1.wav",
    cascade_output_path="output_wav/vc/ar_nar_cascade/train_1.wav",
    nar_output_path="output_wav/vc/nar/train_1.wav",
    seed=0,
    device="cuda"
)

input_dir = "/work/b0990106x/TextRL/data-encodec"
output_path = "/work/b0990106x/TextRL/output/example.wav"

encodec_code = run(args, input_dir, output_path)
print("Done")

Dataset loaded, size is  9957


/home/b0990106x/miniconda3/envs/textrl/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


size of encodec_input torch.Size([1, 8, 653])
type of encodec_input torch.int64
type of the element in encodec_input torch.int64
size of audio 208960
type of audio float32
Mode: cascade AR and NAR
Done


In [3]:
import torch
from datasets import load_from_disk
from transformers import (AutoTokenizer, BartForConditionalGeneration)
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# define path
base_path = '/work/b0990106x/TextRL'
agent_input_dir = f'{base_path}/data-encodec'
agent_output_dir = f'{base_path}/output'
env_input_dir = agent_output_dir
env_output_dir = agent_input_dir
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

device = "cuda" if torch.cuda.is_available() else "cpu"
ar_tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
ar_model = BartForConditionalGeneration.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
nar_model = BartForConditionalGeneration.from_pretrained(nar_checkpoint)
ar_model.to(device)

dataset = load_from_disk(agent_input_dir)
source = dataset[f"src_encodec_0"][0]
instruction = dataset["instruction"][0]
instruction_ids = ar_tokenizer(instruction)["input_ids"][1 : -1]
src_encodec_ids = ar_tokenizer.convert_tokens_to_ids(
    [f"v_tok_{u}" for u in dataset[f"src_encodec_0"][0]])
src_encodec_str = ar_tokenizer.convert_tokens_to_string(
    [f"v_tok_{u}" for u in dataset[f"src_encodec_0"][0]])

In [4]:
all_src_encodec_layers = []
all_src_encodec = []
all_instruction = []
all_instruction_ids = []

# data_len = len(dataset)
data_len = 10
layer_len = 8

for i in range(layer_len):
    all_src_encodec_layers.append(dataset[f"src_encodec_{i}"])

for i in range(data_len):
    src_encodec = []
    for j in range(layer_len):
        src_encodec.append(all_src_encodec_layers[j][i])
    all_src_encodec.append(src_encodec)

for i in range(data_len):
    all_instruction.append(dataset["instruction"][i])
    all_instruction_ids.append(ar_tokenizer(all_instruction[i])["input_ids"][1 : -1])
    

encodec_code:  [[835, 835, 798, 585, 550, 535, 535, 737, 737, 377, 556, 601, 787, 8, 99, 411, 411, 378, 937, 378, 937, 804, 838, 890, 934, 47, 438, 438, 731, 738, 133, 709, 479, 479, 479, 151, 940, 502, 906, 407, 645, 70, 208, 537, 537, 1022, 681, 723, 747, 593, 804, 681, 879, 136, 967, 233, 431, 754, 421, 182, 182, 651, 879, 887, 819, 904, 904, 887, 309, 880, 396, 754, 775, 997, 222, 336, 548, 841, 269, 479, 479, 940, 23, 56, 738, 835, 395, 206, 779, 531, 862, 931, 306, 203, 755, 369, 6, 466, 716, 948, 82, 575, 288, 556, 903, 556, 392, 796, 751, 835, 103, 25, 408, 835, 835, 339, 339, 395, 250, 706, 317, 479, 800, 960, 141, 479, 908, 801, 327, 937, 559, 708, 372, 372, 573, 437, 437, 421, 203, 739, 830, 739, 358, 830, 248, 411, 411, 112, 321, 23, 23, 185, 971, 62, 339, 461, 488, 934, 148, 373, 561, 681, 760, 531, 612, 699, 23, 967, 457, 790, 154, 906, 465, 502, 884, 479, 246, 820, 601, 309, 716, 314, 377, 309, 309, 556, 118, 99, 358, 1018, 862, 779, 62, 835, 25, 254, 254, 677, 73, 143, 

In [5]:
# test the first data
from importlib import reload
import trainer_encodec_vc_inference
reload(trainer_encodec_vc_inference)
from trainer_encodec_vc_inference import get_ar_prediction
from types import SimpleNamespace

args_predict = SimpleNamespace(
    output_path = "/work/b0990106x/TextRL/output/example.wav",
    seed = 0,
    device = "cuda"
)    

single_src_encodec = all_src_encodec[0]
single_instruction = all_instruction[0]
print("single_src_encodec: ", single_src_encodec)
print("single_instruction: ", single_instruction)

decode_ar = get_ar_prediction(args_predict, ar_model, nar_model, ar_tokenizer, nar_tokenizer, single_src_encodec, single_instruction)
decode_ar_ids = ar_tokenizer.convert_tokens_to_ids(
    [f"v_tok_{u}" for u in decode_ar])
decode_ar_str = ar_tokenizer.convert_tokens_to_string(
    [f"v_tok_{u}" for u in decode_ar])
    
print("decode_ar: ", decode_ar)
print("decode_ar_ids: ", decode_ar_ids)
print("decode_ar_str: ", decode_ar_str)


In [ ]:
# inference all data and replace the src_encodec[0] with the decode_ar
all_decode_ar = []
for i in range(data_len):
    print(f"Processing {i}...")
    args_predict.output_path = f"/work/b0990106x/TextRL/output/example_{i}.wav"
    decode_ar = get_ar_prediction(args_predict, ar_model, nar_model, ar_tokenizer, nar_tokenizer, all_src_encodec[i], all_instruction[i])
    # all_src_encodec[i][0] = decode_ar_ids
    all_decode_ar.append(decode_ar)